In [172]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import seaborn as sns 
import collections
from itertools import zip_longest

# 下2個のモジュールはlabelencodingを全体に適用させるためのもの
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [173]:
train_data = pd.read_csv('../input/contradictory-my-dear-watson/train.csv')
train_matrix = train_data.shape[0]

In [174]:
train_data

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [175]:
test_data = pd.read_csv('../input/contradictory-my-dear-watson/test.csv')
test_data.shape

(5195, 5)

In [176]:
data = pd.concat([train_data, test_data], axis=0)

In [177]:
data

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0.0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2.0
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0.0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0.0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1.0
...,...,...,...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,ur,Urdu,NaN
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,en,English,NaN
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,zh,Chinese,NaN
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,en,English,NaN


In [178]:
data.drop(['lang_abv'],1, inplace=True)

In [179]:
# separate sentences

In [180]:
def pre_separate(self):
    num_need_columns = []
    columns_element = collections.OrderedDict()    # 順序を保持する
    for content in self['premise']:
        num_need_columns.append(len(content.split(' ')))
    need_num = max(num_need_columns)
    
    for num in range(need_num):
        columns_element['p_' + '{}'.format(num)] = []
        
    for content in self['premise']:
        for n, word in zip_longest(range(need_num), content.split(' ')):   #zip:多い分の要素が無視される,  zip_longest:足りない分の要素が埋められる
            try:   # 変数値の存在確認
                columns_element['p_' + '{}'.format(n)].append(word)
            except:
                columns_element['p_' + '{}'.format(n)].append(None)
    
    for n in range(need_num):
        self['p_' + '{}'.format(n)] = columns_element['p_' + '{}'.format(n)]
        
    return self

In [181]:
data = pre_separate(data)

In [182]:
c=collections.OrderedDict()
c['P_0']=[]
c['P_0'].append(2)
c

OrderedDict([('P_0', [2])])

In [183]:
data

,id,premise,hypothesis,language,label,p_0,p_1,p_2,p_3,p_4,...,p_186,p_187,p_188,p_189,p_190,p_191,p_192,p_193,p_194,p_195
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,English,0.0,and,these,comments,were,considered,...,None,None,None,None,None,None,None,None,None,None
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,English,2.0,These,are,issues,that,we,...,None,None,None,None,None,None,None,None,None,None
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,French,0.0,Des,petites,choses,comme,celles-là,...,None,None,None,None,None,None,None,None,None,None
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,English,0.0,you,know,they,can't,really,...,None,None,None,None,None,None,None,None,None,None
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,Thai,1.0,ในการเล่นบทบาทสมมุติก็เช่นกัน,โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน,ๆ,อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระ...,ความเชื่อ,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,Urdu,NaN,نیند,نے,وعدہ,کیا,کہ,...,None,None,None,None,None,None,None,None,None,None
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,English,NaN,The,,rock,,has,...,None,None,None,None,None,None,None,None,None,None
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,Chinese,NaN,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,English,NaN,isn't,it,i,can,remember,...,None,None,None,None,None,None,None,None,None,None


In [184]:
def hy_separate(self):
    num_need_columns = []
    columns_element = collections.OrderedDict()    # 順序を保持する
    for content in self['hypothesis']:
        num_need_columns.append(len(content.split(' ')))
    need_num = max(num_need_columns)
    
    for num in range(need_num):
        columns_element['h_' + '{}'.format(num)] = []
        
    for content in self['hypothesis']:
        for n, word in zip_longest(range(need_num), content.split(' ')):   #zip:多い分の要素が無視される,  zip_longest:足りない分の要素が埋められる
            try:   
                columns_element['h_' + '{}'.format(n)].append(word)
            except:
                columns_element['h_' + '{}'.format(n)].append(None)
    
    for n in range(need_num):
        self['h_' + '{}'.format(n)] = columns_element['h_' + '{}'.format(n)]
        
    return self

In [185]:
data = hy_separate(data) 

In [186]:
data

,id,premise,hypothesis,language,label,p_0,p_1,p_2,p_3,p_4,...,h_48,h_49,h_50,h_51,h_52,h_53,h_54,h_55,h_56,h_57
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,English,0.0,and,these,comments,were,considered,...,None,None,None,None,None,None,None,None,None,None
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,English,2.0,These,are,issues,that,we,...,None,None,None,None,None,None,None,None,None,None
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,French,0.0,Des,petites,choses,comme,celles-là,...,None,None,None,None,None,None,None,None,None,None
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,English,0.0,you,know,they,can't,really,...,None,None,None,None,None,None,None,None,None,None
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,Thai,1.0,ในการเล่นบทบาทสมมุติก็เช่นกัน,โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน,ๆ,อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระ...,ความเชื่อ,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,Urdu,NaN,نیند,نے,وعدہ,کیا,کہ,...,None,None,None,None,None,None,None,None,None,None
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,English,NaN,The,,rock,,has,...,None,None,None,None,None,None,None,None,None,None
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,Chinese,NaN,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,English,NaN,isn't,it,i,can,remember,...,None,None,None,None,None,None,None,None,None,None


In [187]:
sep_data = data.copy()

In [188]:
#create dummies variables

In [189]:
outputs_data = sep_data['label']
sep_data.drop(['id', 'premise', 'hypothesis', 'label'],1, inplace=True)

In [190]:
le =  LabelEncoder()

In [191]:
sep_data = sep_data.apply(le.fit_transform)

In [192]:
inputs = sep_data.copy()

In [193]:
def trans_int(self):
    for column in self.columns:
        try:
            self[column] = self[column].astype(int)
        except:
            continue     # 繰り返しに戻る
            
        return self

In [194]:
trans_int(inputs)

,language,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,...,h_48,h_49,h_50,h_51,h_52,h_53,h_54,h_55,h_56,h_57
0,3,2290,3085,1137,3449,1179,1844,1515,2873,1654,...,1,1,1,1,1,1,1,1,1,1
1,3,1914,965,1930,3143,3396,3396,3265,1769,2297,...,1,1,1,1,1,1,1,1,1,1
2,4,616,2590,1095,1138,1092,1550,3092,1237,3198,...,1,1,1,1,1,1,1,1,1,1
3,3,2575,2060,3147,1060,2721,1252,2939,1990,2605,...,1,1,1,1,1,1,1,1,1,1
4,11,4690,5715,5785,5681,5508,5518,5319,5115,5001,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,13,3868,5021,5182,5247,5190,5008,4844,4495,4555,...,1,1,1,1,1,1,1,1,1,1
5191,3,1906,0,2814,0,1736,710,2768,2864,742,...,1,1,1,1,1,1,1,1,1,1
5192,2,4923,5770,5805,5746,5611,5543,5319,5115,5001,...,1,1,1,1,1,1,1,1,1,1
5193,3,2402,1967,1821,1059,2755,1810,2340,865,1525,...,1,1,1,1,1,1,1,1,1,1


In [195]:
inputs

,language,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,...,h_48,h_49,h_50,h_51,h_52,h_53,h_54,h_55,h_56,h_57
0,3,2290,3085,1137,3449,1179,1844,1515,2873,1654,...,1,1,1,1,1,1,1,1,1,1
1,3,1914,965,1930,3143,3396,3396,3265,1769,2297,...,1,1,1,1,1,1,1,1,1,1
2,4,616,2590,1095,1138,1092,1550,3092,1237,3198,...,1,1,1,1,1,1,1,1,1,1
3,3,2575,2060,3147,1060,2721,1252,2939,1990,2605,...,1,1,1,1,1,1,1,1,1,1
4,11,4690,5715,5785,5681,5508,5518,5319,5115,5001,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,13,3868,5021,5182,5247,5190,5008,4844,4495,4555,...,1,1,1,1,1,1,1,1,1,1
5191,3,1906,0,2814,0,1736,710,2768,2864,742,...,1,1,1,1,1,1,1,1,1,1
5192,2,4923,5770,5805,5746,5611,5543,5319,5115,5001,...,1,1,1,1,1,1,1,1,1,1
5193,3,2402,1967,1821,1059,2755,1810,2340,865,1525,...,1,1,1,1,1,1,1,1,1,1


In [196]:
inputs_train = inputs[:train_matrix]
inputs_test = inputs[train_matrix:]

In [197]:
outputs_train = outputs_data[:train_matrix]

In [198]:
outputs_train

0        0.0
1        2.0
2        0.0
3        0.0
4        1.0
        ... 
12115    2.0
12116    0.0
12117    2.0
12118    2.0
12119    0.0
Name: label, Length: 12120, dtype: float64

In [199]:
x_train, x_val, y_train, y_val = train_test_split(inputs_train, outputs_train, test_size=0.85)

In [200]:
gnb = GaussianNB().fit(x_train, y_train)
knnc = KNeighborsClassifier().fit(x_train, y_train)
cartc = DecisionTreeClassifier(random_state=42).fit(x_train, y_train)
rfc = RandomForestClassifier(random_state=42, verbose=False).fit(x_train, y_train)
gbmc = GradientBoostingClassifier(verbose=False).fit(x_train, y_train)
xgbc = XGBClassifier().fit(x_train, y_train)
lgbmc = LGBMClassifier().fit(x_train, y_train)
catbc = CatBoostClassifier(verbose=False).fit(x_train, y_train)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:25:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [ ]:
modelsc = [gnb, knnc, cartc, rfc, gbmc, lgbmc, catbc]

In [ ]:
"""
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(x_val)
    R2CV = cross_val_score(model, x_val, y_val, cv=10, verbose=False).mean()
    error = -cross_val_score(model, x_val, y_val, cv=10, scoring='neg_mean_squared_error', verbose=False).mean()
    print(name+':')
    print('-------'*10)
    print(accuracy_score(y_val, predict))
    print('-------'*10)
    print(R2CV)
    print(np.sqrt(error))
    print('-'*30)
"""

In [201]:
# 上記の結果より、勾配ブースティング分類法を用いる
gbmc2 = GradientBoostingClassifier(verbose=False).fit(inputs_train, outputs_train)

In [202]:
pre_test = gbmc2.predict(inputs_test).astype(int)

In [203]:
# create submission
sub_df = pd.read_csv('../input/contradictory-my-dear-watson/sample_submission.csv')
sub_df['prediction'] = pre_test
sub_df.to_csv('submission.csv', index=False)

In [209]:
pre_test.astype(int)

array([2, 1, 2, ..., 0, 2, 1])

In [208]:
sub_df

,id,prediction
0,c6d58c3f69,2.0
1,cefcc82292,1.0
2,e98005252c,2.0
3,58518c10ba,0.0
4,c32b0d16df,1.0
...,...,...
5190,5f90dd59b0,1.0
5191,f357a04e86,1.0
5192,1f0ea92118,0.0
5193,0407b48afb,2.0
